In [13]:
import numpy as np
import deepSI
from deepSI import System_data

out= np.load('training-data.npz')
th_train = out['th']
u_train = out['u']

data = np.load('test-simulation-submission-file.npz')
u_test = data['u']
th_test = data['th']

system_data = System_data(u=u_train,y=th_train)

if False: #train model
    sys = deepSI.fit_systems.SS_encoder(nx=13,na=2,nb=3) #load the encoder system
    sys.n_hidden_layers = 1
    sys.n_nodes_per_layer = 20
    sys.fit(system_data, val_sys_data=system_data[-12_000:], epochs=500, batch_size=1024, loss_kwargs=dict(nf=40)) 
    sys.save_system('Model_encoder\\encoder')

    
    sim_val = system_data[-25_000:] #larger validation set
    sys.bestfit = sys.apply_experiment(sim_val).NRMS(sim_val) #reset checkpoint
    #use all the data for final local minima search
    sys.fit(system_data,sim_val, epochs=200 ,batch_size=1024, loss_kwargs=dict(nf=40)) 
    sys.save_system('Model_encoder\\encoder_final')
    
else: #load model from file
    sys = deepSI.load_system('Model_encoder\encoder_final')


train_simulation = sys.apply_experiment(system_data)

print('train simulation errors:')
print('RMS:', train_simulation.RMS(system_data),'radians')
print('RMS:', train_simulation.RMS(system_data)/(2*np.pi)*360,'degrees')
print('NRMS:', train_simulation.NRMS(system_data),'%')


u_test1 =  u_test[37:]
th_test1= th_test[37:]

test_system_data = System_data(u=u_test1,y=th_test1)
test_sim = sys.apply_experiment(test_system_data)
th_test_sim = np.append(th_test[0:50],test_sim.y[13:])
np.savez('test_simulation-submission-file.npz', th=th_test_sim,u=u_test)




train simulation errors:
RMS: 0.009333977556371711 radians
RMS: 0.5347975200499325 degrees
NRMS: 0.019030714938085315 %
